In [5]:
import pandas as pd
import numpy as np

**Load .trc file for first walking trial**

In [40]:
# --- FILE PATHS ---
input_file = "/Users/briankeller/Desktop/GRFMuscleModel/Old Young Walking Data/OA1/Walking/OA1_80_1.trc"
output_file = "/Users/briankeller/Desktop/GRFMuscleModel/Old Young Walking Data/OA1/Walking/OA1_80_1_rotated.trc"

# --- LOAD HEADER & DATA ---
with open(input_file, 'r') as f:
    header_lines = [next(f) for _ in range(4)]  # First 5 lines are header
    df = pd.read_csv(f, sep='\t')  # Data starts from line 6

# --- SAVE A BACKUP COPY ---
df_orig = df.copy()

# --- SPLIT META (Frame# and Time) ---
print(df.columns)
meta_cols = df.columns[:2]  # Safely grabs first two columns: Frame# and Time
meta_df = df[meta_cols]
df = df.drop(columns=meta_cols)
meta_df = meta_df.rename(columns={meta_df.columns[0]: '', meta_df.columns[1]: ''})
print(df.columns)

# --- DEFINE ROTATION (+90° about X-axis) ---
angle = np.radians(-90)
R = np.array([
    [1, 0, 0],
    [0, np.cos(angle), -np.sin(angle)],
    [0, np.sin(angle),  np.cos(angle)]
])

# --- ROTATE MARKER COORDINATES ---
marker_ids = sorted(set(int(col[1:]) for col in df.columns if col.startswith('X')))
print(marker_ids)
for idx in marker_ids:
    try:
        coords = df[[f'X{idx}', f'Y{idx}', f'Z{idx}']].astype(float).values
        rotated = coords @ R.T
        df[f'X{idx}'], df[f'Y{idx}'], df[f'Z{idx}'] = rotated[:, 0], rotated[:, 1], rotated[:, 2]
    except KeyError:
        continue

# --- RECOMBINE AND SAVE ---
df_final = pd.concat([meta_df, df], axis=1)
print(df_final.head)

with open(output_file, 'w') as f:
    for line in header_lines:
        f.write(line)
    df_final.to_csv(f, sep='\t', index=False, lineterminator='\n')

Index(['Unnamed: 0', 'Unnamed: 1', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'X3',
       'Y3',
       ...
       'X40', 'Y40', 'Z40', 'X41', 'Y41', 'Z41', 'X42', 'Y42', 'Z42',
       'Unnamed: 128'],
      dtype='object', length=129)
Index(['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'X3', 'Y3', 'Z3', 'X4',
       ...
       'X40', 'Y40', 'Z40', 'X41', 'Y41', 'Z41', 'X42', 'Y42', 'Z42',
       'Unnamed: 128'],
      dtype='object', length=127)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
<bound method NDFrame.head of                         X1          Y1        Z1          X2          Y2  \
0      1  0.00 -1542.15417  1388.59204 -12.97525 -1523.77502  1339.45776   
1      2  0.01 -1531.35962  1386.64551 -15.02223 -1513.36902  1337.38940   
2      3  0.02 -1521.07678  1384.84424 -16.24288 -1502.35950  1335.26563   
3      4  0.03 -1510.41602  1383.37463 -18.16332 -1492.25012  1333.7929